In [269]:
import requests
from bs4 import BeautifulSoup
import time
import re

url="https://www.immoweb.be/en/search/house/for-sale/gent/district?countries=BE&priceType=SALE_PRICE&minBedroomCount=8&page=1&orderBy=relevance" 
headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

r= requests.get(url,headers=headers)
soup=BeautifulSoup(r.content, 'html.parser')
links=soup.find_all("a", class_="card__title-link")

for link in links:
    house_url=link.get("href")
    house_response=requests.get(house_url, headers=headers)
    house_soup=BeautifulSoup(house_response.text, 'html.parser')

# Property Type 
    try:
        house_type= house_soup.find('h1', class_='classified__title')
        house_type_text=kitchen.text
        cleaned_title = house_type_text.replace("for sale", "").strip()
        print(cleaned_title)  
    except:
        print(None)  

# Property ID
    immoweb_code=house_soup.find('div', class_='classified__header--immoweb-code')
    code=immoweb_code.text.split(':')
    property_id=code[1].strip()
    print(f"Property ID : {property_id}")

# Price
    try:
        price_elements = house_soup.find('span', class_='sr-only')  
        text= price_elements.text
        prices = text.replace('From ', '').replace('To ', '').replace('€', '').split()
        price_range = '-'.join(prices)
        print(price_range)
    except:
        print("Price information is not found")

# Number of Bedrooms
    try:
        info=house_soup.find('p', class_="classified__information--property")
        parts = info.text.split('|')  # Metni ayırıcı olarak '|' kullanarak ikiye böl
        bedrooms = parts[0].strip()  # İlk kısmı (bedrooms) al
        print(f"Bedrooms : {bedrooms}")
    except IndexError:
        print("Bedroom information is not found")

# Living Area
    try:
        info=house_soup.find('p', class_="classified__information--property")
        parts = info.text.split('|') 
        area = parts[1].strip() 
        area_cleaned = ' '.join(area.split())
        print(f"Living area: {area_cleaned}")
    except IndexError:
        print("Living area information is not available.")

# Kitchen Type
    try:
        kitchen= house_soup.find('th', string='Kitchen type').find_next_sibling('td').contents[0].strip()
        print(f"Kitchen Type : {kitchen}")
    except:
        print("Kitchen type information is not available")

# Terrace
    try:
        terrace=house_soup.find('th', string=['Terrace', 'Terrace surface']).find_next_sibling('td').contents[0].strip() 
        print(f"Terrace : {terrace} m²")
    except:
        print("Terrace information is not available")

# Garden
    try:
        garden=house_soup.find('th', string=['Garden surface', 'Garden surface']).find_next_sibling('td').contents[0].strip() 
        print(f"Garden : {garden} m²")
    except:
        print("Garden information is not available ")

# Furnished
    try:
        furnish=house_soup.find('th', string='Furnished').find_next_sibling('td').contents[0].strip() 
        print(f"Furnished : {furnish} ")
    except:
        print("Furnishing information is not available")

        



Property ID : 20221403
Price : 1500000€
Bedrooms : 11 bedrooms
Living area: 770 m² square meters
Kitchen Type : Hyper equipped
Terrace : 70 m²
Garden information is not available 
Furnished : No 
Property ID : 20226411
Price : 350000€
Bedrooms : 9 bedrooms
Living area: 300 m² square meters
Kitchen type information is not available
Terrace information is not available
Garden information is not available 
Furnishing information is not available
Property ID : 20099588
Price : 1150000€
Bedrooms : 15 bedrooms
Living area: 950 m² square meters
Kitchen Type : Installed
Terrace information is not available
Garden : 23 m²
Furnishing information is not available
Property ID : 11443019
Price : 1595000€
Bedrooms : 8 bedrooms
Living area: 468 m² square meters
Kitchen Type : Installed
Terrace : Yes m²
Garden information is not available 
Furnishing information is not available
Property ID : 11453441
Price : 1380000€
Bedrooms : 9 bedrooms
Living area: 240 m² square meters
Kitchen Type : Semi equipped

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

url="https://www.immoweb.be/en/search/house/for-sale?countries=BE&minBedroomCount=5&page=1&orderBy=relevance" 
headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

r= requests.get(url,headers=headers)
soup=BeautifulSoup(r.content, 'html.parser')
links=soup.find_all("a", class_="card__title-link")

listings = []

for link in links:
    house_url=link.get("href")
    house_response=requests.get(house_url, headers=headers)
    house_soup=BeautifulSoup(house_response.text, 'html.parser')

    house_data= {}


# Property Type 
    try:
        house_type= house_soup.find('h1', class_='classified__title')
        cleaned_title = house_type.text.replace("for sale", "").strip()
        house_data['Property Type'] = cleaned_title  
    except:
        house_data['Property Type'] = 'None' 

# Property ID
    try:
        immoweb_code = house_soup.find('div', class_='classified__header--immoweb-code')
        code = immoweb_code.text.split(':')
        house_data['Property ID'] = code[1].strip()
    except:
        house_data['Property ID'] = 'Not found'
    

# Price
    try:
        price_elements = house_soup.find('span', class_='sr-only')
        text= price_elements.text
        prices = text.replace('From ', '').replace('To ', '').replace('€', '').split()
        price_range = '-'.join(prices)
        house_data['Price'] = price_range.strip()
    except:
        house_data['Price'] = 'Not found'
        

# Number of Bedrooms
    try:
        info = house_soup.find('p', class_="classified__information--property")
        parts = info.text.split('|')
        bedrooms = parts[0].strip()  # İlk kısmı (bedrooms) al
        no=bedrooms[0:2]
        house_data['Bedrooms'] = no
    except (IndexError, AttributeError):
        house_data['Bedrooms'] = 'Not found'
        
# Living Area
    try:
        info = house_soup.find('p', class_="classified__information--property")
        parts = info.text.split('|')
        area = parts[1].strip()
        are=area.split()[0]
        house_data['Living Area'] = are
    except (IndexError, AttributeError):
        house_data['Living Area'] = 'Not found'

# Kitchen Type
    try:
        kitchen = house_soup.find('th', string='Kitchen type').find_next_sibling('td').contents[0].strip()
        house_data['Kitchen Type'] = kitchen
    except:
        house_data['Kitchen Type'] = 'Not found'

# Terrace
    try:
        terrace=house_soup.find('th', string=['Terrace', 'Terrace surface']).find_next_sibling('td').contents[0].strip() 
        house_data['Terrace'] = terrace
    except:
        house_data['Terrace'] = 'Not found'

# Garden
    try:
        garden=house_soup.find('th', string=['Garden surface', 'Garden surface']).find_next_sibling('td').contents[0].strip() 
        house_data['Garden'] = garden
    except:
        house_data['Garden'] = 'Not found'

# Furnished
    try:
        furnish=house_soup.find('th', string='Furnished').find_next_sibling('td').contents[0].strip() 
        house_data['Furnished'] = furnish
    except:
         house_data['Furnished'] = 'Not found'      

    listings.append(house_data)

 # İlanları bir DataFrame'e çevir        
df = pd.DataFrame(listings)

# DataFrame'i CSV dosyasına kaydet
csv_file_path = 'house_listings.csv'  # CSV dosya ismi
df.to_csv(csv_file_path, index=False)  # index=False ile indeks sütunu eklenmez

print(f"Data has been written to {csv_file_path}")


Data has been written to house_listings.csv


In [7]:
import requests
from bs4 import BeautifulSoup
import time
import re

url="https://www.immoweb.be/en/classified/mixed-use-building/for-sale/gent/9000/20221403"
headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

r= requests.get(url,headers=headers)

soup=BeautifulSoup(r.content, 'html.parser')
links=soup.find_all("a", class_="card__title-link")

# Swimming Pool

swimming_pool=soup.find('th', string=re.compile(r'\swimming pool\s'))#.find_next_sibling('td').contents[0].strip()
print(f"Swimming Pool : {swimming_pool} ")





Swimming Pool : None 


In [5]:
import requests
from bs4 import BeautifulSoup
import time
url="https://www.immoweb.be/en/search/house/for-sale/gent/district?countries=BE&priceType=SALE_PRICE&minBedroomCount=8&page=1&orderBy=relevance" 
headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

r= requests.get(url,headers=headers)
soup=BeautifulSoup(r.content, 'html.parser')
links=soup.find_all("a", class_="card__title-link")

for link in links:
    house_url=link.get("href")
    house_response=requests.get(house_url, headers=headers)
    house_soup=BeautifulSoup(house_response.text, 'html.parser')


# Building Condition 
    try:
        condition=house_soup.find('th', string='Building contition')#.find_next_sibling('td').contents[0].strip()
        print(condition)
    except:
        print(None)

Swimming_pool = soup.find('th', string=re.compile(r'\swimming pool\s')).find_next_sibling('td').contents[0].strip()


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [175]:
#Location

import requests
from bs4 import BeautifulSoup
import time

url="https://www.immoweb.be/en/search/house/for-sale/gent/district?countries=BE&priceType=SALE_PRICE&minBedroomCount=8&page=1&orderBy=relevance" 
headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

r= requests.get(url,headers=headers)
soup=BeautifulSoup(r.content, 'html.parser')
links=soup.find_all("a", class_="card__title-link")

#for link in links:
    #print(link.get('href'))

# Diğer sayfalardaki immoweb kodlarını bulmak için bir döngü
for link in links:
    houses_url=link.get("href")
    houses_response=requests.get(house_url, headers=headers)
    house_soup=BeautifulSoup(house_response.text, 'html.parser')

   

    house_link = house_soup.find('a', class_="card__title-link")  # <a> tag'ini bul
    print(house_link)
    
    if house_link and house_link.has_attr('aria-label'):  # Eğer aria-label varsa
        aria_label = house_link['aria-label']  # aria-label özelliğini al
        city = aria_label.split(',')[1].split('(')[0].strip()  # "Gent" bilgisini çek
        #print(f"City: {city}")  # Şehir bilgisini yazdır
    else:
       # print("City information not found.")

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [130]:
#   Living area ✔️ 

import requests
from bs4 import BeautifulSoup
import time

url="https://www.immoweb.be/en/search/house/for-sale/gent/district?countries=BE&priceType=SALE_PRICE&minBedroomCount=8&page=1&orderBy=relevance" 
headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

r= requests.get(url,headers=headers)
soup=BeautifulSoup(r.content, 'html.parser')
links=soup.find_all("a", class_="card__title-link")

#for link in links:
    #print(link.get('href'))

# Diğer sayfalardaki immoweb kodlarını bulmak için bir döngü
for link in links:
    house_url=link.get("href")
    house_response=requests.get(house_url, headers=headers)
    house_soup=BeautifulSoup(house_response.text, 'html.parser')

    immoweb_code=house_soup.find('div', class_='classified__header--immoweb-code')
    code=immoweb_code.text.split(':')
    property_id=code[1].strip()
    #print(f"Property ID : {property_id}")
    
    info=house_soup.find('p', class_="classified__information--property")
    parts = info.text.split('|')  # Metni ayırıcı olarak '|' kullanarak ikiye böl
    bedrooms = parts[0].strip()  # İlk kısmı (bedrooms) al
    #print(f"Bedrooms : {bedrooms}")

    try:
        info=house_soup.find('p', class_="classified__information--property")
        parts = info.text.split('|') 
        area = parts[1].strip() 
        area_cleaned = ' '.join(area.split())
        #print(f"Living area: {area_cleaned}")
    except IndexError:
        #print("Living area information is not available.")




IndentationError: expected an indented block (4285850041.py, line 42)

In [298]:
# Price ✔️ 

import requests
from bs4 import BeautifulSoup
import time

url="https://www.immoweb.be/en/search/house/for-sale?countries=BE&minBedroomCount=6&page=1&orderBy=relevance" 
headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

r= requests.get(url,headers=headers)
soup=BeautifulSoup(r.content, 'html.parser')
links=soup.find_all("a", class_="card__title-link")

#for link in links:
    #print(link.get('href'))

# Diğer sayfalardaki immoweb kodlarını bulmak için bir döngü
for link in links:
    house_url=link.get("href")
    house_response=requests.get(house_url, headers=headers)
    house_soup=BeautifulSoup(house_response.text, 'html.parser')

   

    try:
        price_elements = house_soup.find('span', class_='sr-only')  
        text= price_elements.text
        prices = text.replace('From ', '').replace('To ', '').replace('€', '').split()
        price_range = '-'.join(prices)
        print(price_range)
    except:
        print("Price information is not found")



1200000-1375000
1200000-1375000
975000
975000
975000
549000
549000
790000
249000
249000
1150000
298500
465000
1200000-1375000
424000
465000
850000
665000
850000
965000
240000
529000
529000
650000
1150000
999900
465000
600000
600000
600000
1190000
415000
429000
340000
340000
1445000
249000
249000
329000
898000
265000
265000
524000
1590000
249000
1995000
329000
185000
399000
275000
625000
975000
429000
815000
329000
675000
990000
469000
875000
790000


In [102]:
#Property ID Sayisi=51
import requests
from bs4 import BeautifulSoup
import time

url="https://www.immoweb.be/en/search/house/for-sale/gent/district?countries=BE&priceType=SALE_PRICE&minBedroomCount=8&page=1&orderBy=relevance" 
headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

r= requests.get(url,headers=headers)
soup=BeautifulSoup(r.content, 'html.parser')
links=soup.find_all("a", class_="card__title-link")

#for link in links:
    #print(link.get('href'))

# Diğer sayfalardaki immoweb kodlarını bulmak için bir döngü
property_ids = []
for link in links:
    house_url=link.get("href")
    house_response=requests.get(house_url, headers=headers)
    house_soup=BeautifulSoup(house_response.text, 'html.parser')

    immoweb_code=house_soup.find('div', class_='classified__header--immoweb-code')
    code=immoweb_code.text.split(':')
    property_id=code[1].strip()
    #print(f"Property ID : {property_id}")
    property_ids.append(property_id)
    number_of_property_ids = len(property_ids)
    #print(number_of_property_ids)

    

Living area information is not available.


In [ ]:
import requests
from bs4 import BeautifulSoup
import time


url="https://www.immoweb.be/en/search/house/for-sale/gent/district?countries=BE&minBedroomCount=8&page=1&orderBy=newest"
headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

page= requests.get(url,headers=headers)
soup=BeautifulSoup(page.content, 'html.parser')
    
    
all_houses=soup.find_all("li", class_="search-results__item")
    
for house in all_houses:
    item={}
    item['Price']=house.find("a").attrs["aria-label"]

    print(item['Price'])
    
   

            #item['propertyID']= house.find("div", class_="classified__header--immoweb-code")

        #print(item['propertyID'])
   


AttributeError: 'NoneType' object has no attribute 'attrs'

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

current_page=1
proceed= True

while proceed:
    url="https://www.immoweb.be/en/search/house/for-sale/gent/district?countries=BE&minBedroomCount=8&page=1&orderBy=newest"
    headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

    page= requests.get(url,headers=headers)
    soup=BeautifulSoup(page.content, 'html.parser')

    if soup.title.text=="404 Not Found":
        proceed=False

    else:
    
        all_houses=soup.find_all("li", class_="search-results__item")
        print(all_houses)
    
        for house in all_houses:
            item={}
            #item['Price']=house.find("a").attrs["aria-label"]
            item['Bedrooms']=house.find("p", class_="classified__information--property")


    #print(item['Price'])
    print(item['Bedrooms'])

    current_page +=1
    proceed=False

In [18]:
import requests
from bs4 import BeautifulSoup
import time



url="https://www.immoweb.be/en/classified/mixed-use-building/for-sale/gent/9000/20221403"
headers= {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

page= requests.get(url,headers=headers)
soup=BeautifulSoup(page.content, 'html.parser')

#all_houses=soup.find_all("li", class_="search-results__item")
#for house in all_houses:
    #item={}

    #item['propertyID']= house.find("div", class_="classified__header--immoweb-code")

    #print(item['propertyID'])
   
home_meta_info = soup.find_all("div", {'class': 'grid__item desktop--9'})
Property_ID = home_meta_info[0].find_all("div", {'class': 'classified__header-secondary-info'})[0].find_all("div", {'class': 'classified__header--immoweb-code'})[0].text.strip()
Property_ID = Property_ID.split(':')[-1].strip()   # to avoid returnning the whole " Immoweb code : 20216274"


print(Property_ID)



20221403
